# 나이브 베이즈 분류기를 사용하여 League of Legends 승패 예측하기

## 필요한 라이브러리 불러오기

In [12]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn import metrics

## League of Legends 데이터 불러오고 살펴보기

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [21]:
lol_df = pd.read_csv('/content/drive/My Drive/kaggle/LeagueofLegends/high_diamond_ranked_10min.csv')
print(type(lol_df))
lol_df

<class 'pandas.core.frame.DataFrame'>


,gameId,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,blueTowersDestroyed,blueTotalGold,blueAvgLevel,blueTotalExperience,blueTotalMinionsKilled,blueTotalJungleMinionsKilled,blueGoldDiff,blueExperienceDiff,blueCSPerMin,blueGoldPerMin,redWardsPlaced,redWardsDestroyed,redFirstBlood,redKills,redDeaths,redAssists,redEliteMonsters,redDragons,redHeralds,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,4519157822,0,28,2,1,9,6,11,0,0,0,0,17210,6.6,17039,195,36,643,-8,19.5,1721.0,15,6,0,6,9,8,0,0,0,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,4523371949,0,12,1,0,5,5,5,0,0,0,0,14712,6.6,16265,174,43,-2908,-1173,17.4,1471.2,12,1,1,5,5,2,2,1,1,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,4521474530,0,15,0,0,7,11,4,1,1,0,0,16113,6.4,16221,186,46,-1172,-1033,18.6,1611.3,15,3,1,11,7,14,0,0,0,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,4524384067,0,43,1,0,4,5,5,1,0,1,0,15157,7.0,17954,201,55,-1321,-7,20.1,1515.7,15,2,1,5,4,10,0,0,0,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,4436033771,0,75,4,0,6,6,6,0,0,0,0,16400,7.0,18543,210,57,-1004,230,21.0,1640.0,17,2,1,6,6,7,1,1,0,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9874,4527873286,1,17,2,1,7,4,5,1,1,0,0,17765,7.2,18967,211,69,2519,2469,21.1,1776.5,46,3,0,4,7,7,0,0,0,0,15246,6.8,16498,229,34,-2519,-2469,22.9,1524.6
9875,4527797466,1,54,0,0,6,4,8,1,1,0,0,16238,7.2,19255,233,48,782,888,23.3,1623.8,12,21,1,4,6,3,0,0,0,0,15456,7.0,18367,206,56,-782,-888,20.6,1545.6
9876,4527713716,0,23,1,0,6,7,5,0,0,0,0,15903,7.0,18032,210,45,-2416,-1877,21.0,1590.3,14,0,1,7,6,11,1,1,0,0,18319,7.4,19909,261,60,2416,1877,26.1,1831.9
9877,4527628313,0,14,4,1,2,3,3,1,1,0,0,14459,6.6,17229,224,48,-839,-1085,22.4,1445.9,66,4,0,3,2,1,0,0,0,0,15298,7.2,18314,247,40,839,1085,24.7,1529.8


In [36]:
lol_X = lol_df.loc[:,'blueWardsPlaced':'redGoldPerMin']
lol_X

,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,blueTowersDestroyed,blueTotalGold,blueAvgLevel,blueTotalExperience,blueTotalMinionsKilled,blueTotalJungleMinionsKilled,blueGoldDiff,blueExperienceDiff,blueCSPerMin,blueGoldPerMin,redWardsPlaced,redWardsDestroyed,redFirstBlood,redKills,redDeaths,redAssists,redEliteMonsters,redDragons,redHeralds,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,28,2,1,9,6,11,0,0,0,0,17210,6.6,17039,195,36,643,-8,19.5,1721.0,15,6,0,6,9,8,0,0,0,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,12,1,0,5,5,5,0,0,0,0,14712,6.6,16265,174,43,-2908,-1173,17.4,1471.2,12,1,1,5,5,2,2,1,1,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,15,0,0,7,11,4,1,1,0,0,16113,6.4,16221,186,46,-1172,-1033,18.6,1611.3,15,3,1,11,7,14,0,0,0,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,43,1,0,4,5,5,1,0,1,0,15157,7.0,17954,201,55,-1321,-7,20.1,1515.7,15,2,1,5,4,10,0,0,0,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,75,4,0,6,6,6,0,0,0,0,16400,7.0,18543,210,57,-1004,230,21.0,1640.0,17,2,1,6,6,7,1,1,0,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9874,17,2,1,7,4,5,1,1,0,0,17765,7.2,18967,211,69,2519,2469,21.1,1776.5,46,3,0,4,7,7,0,0,0,0,15246,6.8,16498,229,34,-2519,-2469,22.9,1524.6
9875,54,0,0,6,4,8,1,1,0,0,16238,7.2,19255,233,48,782,888,23.3,1623.8,12,21,1,4,6,3,0,0,0,0,15456,7.0,18367,206,56,-782,-888,20.6,1545.6
9876,23,1,0,6,7,5,0,0,0,0,15903,7.0,18032,210,45,-2416,-1877,21.0,1590.3,14,0,1,7,6,11,1,1,0,0,18319,7.4,19909,261,60,2416,1877,26.1,1831.9
9877,14,4,1,2,3,3,1,1,0,0,14459,6.6,17229,224,48,-839,-1085,22.4,1445.9,66,4,0,3,2,1,0,0,0,0,15298,7.2,18314,247,40,839,1085,24.7,1529.8


In [38]:
lol_y = lol_df.loc[:,'blueWins']
lol_y

0       0
1       0
2       0
3       0
4       0
       ..
9874    1
9875    1
9876    0
9877    0
9878    1
Name: blueWins, Length: 9879, dtype: int64

## train_test_split()

In [39]:
X_train, X_test, y_train, y_test = train_test_split(lol_X, lol_y, test_size=0.2)

In [40]:
X_train.describe()

,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,blueTowersDestroyed,blueTotalGold,blueAvgLevel,blueTotalExperience,blueTotalMinionsKilled,blueTotalJungleMinionsKilled,blueGoldDiff,blueExperienceDiff,blueCSPerMin,blueGoldPerMin,redWardsPlaced,redWardsDestroyed,redFirstBlood,redKills,redDeaths,redAssists,redEliteMonsters,redDragons,redHeralds,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
count,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.00000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.00000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000,7903.000000
mean,22.293306,2.827281,0.503606,6.198912,6.132102,6.677464,0.552195,0.361635,0.190561,0.050867,16511.485006,6.918208,17934.301531,216.636467,50.545742,26.028976,-18.553334,21.663647,1651.148501,22.31836,2.713780,0.496394,6.132102,6.198912,6.652284,0.566241,0.409971,0.156270,0.04416,16485.456029,6.923649,17952.854865,217.373909,51.234215,-26.028976,18.553334,21.737391,1648.545603
std,17.988544,2.210203,0.500019,3.015217,2.941041,4.090238,0.627173,0.480504,0.392768,0.245834,1534.481817,0.305010,1203.113041,21.945546,9.881500,2456.333297,1917.971708,2.194555,153.448182,18.52223,2.144093,0.500019,2.941041,3.015217,4.050967,0.626855,0.491859,0.363134,0.21975,1499.004614,0.306475,1201.099662,21.925786,10.025653,2456.333297,1917.971708,2.192579,149.900461
min,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10730.000000,4.600000,10098.000000,90.000000,0.000000,-10830.000000,-9333.000000,9.000000,1073.000000,6.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,11212.000000,4.800000,10465.000000,107.000000,4.000000,-11467.000000,-8348.000000,10.700000,1121.200000
25%,14.000000,1.000000,0.000000,4.000000,4.000000,4.000000,0.000000,0.000000,0.000000,0.000000,15419.000000,6.800000,17168.000000,202.000000,44.000000,-1595.000000,-1275.000000,20.200000,1541.900000,14.00000,1.000000,0.000000,4.000000,4.000000,4.000000,0.000000,0.000000,0.000000,0.00000,15426.500000,6.800000,17198.000000,203.000000,44.000000,-1644.500000,-1239.000000,20.300000,1542.650000
50%,16.000000,3.000000,1.000000,6.000000,6.000000,6.000000,0.000000,0.000000,0.000000,0.000000,16417.000000,7.000000,17965.000000,218.000000,50.000000,34.000000,-9.000000,21.800000,1641.700000,16.00000,2.000000,0.000000,6.000000,6.000000,6.000000,0.000000,0.000000,0.000000,0.00000,16371.000000,7.000000,17970.000000,218.000000,51.000000,-34.000000,9.000000,21.800000,1637.100000
75%,20.000000,4.000000,1.000000,8.000000,8.000000,9.000000,1.000000,1.000000,0.000000,0.000000,17474.000000,7.200000,18731.500000,232.000000,56.000000,1644.500000,1239.000000,23.200000,1747.400000,20.00000,4.000000,1.000000,8.000000,8.000000,9.000000,1.000000,1.000000,0.000000,0.00000,17426.000000,7.200000,18753.000000,233.000000,57.000000,1595.000000,1275.000000,23.300000,1742.600000
max,250.000000,27.000000,1.000000,19.000000,22.000000,29.000000,2.000000,1.000000,1.000000,4.000000,23701.000000,8.000000,22125.000000,283.000000,85.000000,11467.000000,8348.000000,28.300000,2370.100000,276.00000,24.000000,1.000000,22.000000,19.000000,28.000000,2.000000,1.000000,1.000000,2.00000,22732.000000,8.200000,22269.000000,289.000000,92.000000,10830.000000,9333.000000,28.900000,2273.200000


In [41]:
y_train.describe()

count    7903.000000
mean        0.503733
std         0.500018
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: blueWins, dtype: float64

## 데이터 전처리 하기

In [42]:
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

## 가우시안 나이브 베이즈

* 입력 특성이 가우시안(정규) 분포를 갖는다고 가정

### 전처리를 거치지 않은 데이터

In [43]:
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [44]:
predict = model.predict(X_train)
acc = metrics.accuracy_score(y_train, predict)
f1 = metrics.f1_score(y_train, predict, average=None)

print("Train Accuracy: {}".format(acc))
print("Train F1 score {}".format(f1))

Train Accuracy: 0.7247880551689232
Train F1 score [0.72485769 0.72471839]


In [45]:
predict = model.predict(X_test)
acc = metrics.accuracy_score(y_test, predict)
f1 = metrics.f1_score(y_test, predict, average=None)

print("Train Accuracy: {}".format(acc))
print("Train F1 score {}".format(f1))

Train Accuracy: 0.7388663967611336
Train F1 score [0.74853801 0.72842105]


### 전처리 후 데이터

In [52]:
model = GaussianNB()
model.fit(X_train_scale, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [53]:
predict = model.predict(X_train_scale)
acc = metrics.accuracy_score(y_train, predict)
f1 = metrics.f1_score(y_train, predict, average=None)

print("Train Accuracy: {}".format(acc))
print("Train F1 score {}".format(f1))

Train Accuracy: 0.7246615209414147
Train F1 score [0.72462668 0.72469636]


In [54]:
predict = model.predict(X_test_scale)
acc = metrics.accuracy_score(y_test, predict)
f1 = metrics.f1_score(y_test, predict, average=None)

print("Train Accuracy: {}".format(acc))
print("Train F1 score {}".format(f1))

Train Accuracy: 0.7373481781376519
Train F1 score [0.74769081 0.72612137]


## 베르누이 나이브 베이즈

* 입력 특성이 베르누이 분포에 의해 생성된 이진 값을 갖는 다고 가정

### 전처리를 거치지 않은 데이터

In [55]:
model = BernoulliNB()
model.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [56]:
predict = model.predict(X_train)
acc = metrics.accuracy_score(y_train, predict)
f1 = metrics.f1_score(y_train, predict, average=None)

print('Train Accuracy: {}'.format(acc))
print('Train F1 Score: {}'.format(f1))

Train Accuracy: 0.7235227128938377
Train F1 Score: [0.72232812 0.72470707]


In [57]:
predict = model.predict(X_test)
acc = metrics.accuracy_score(y_test, predict)
f1 = metrics.f1_score(y_test, predict, average=None)

print('Train Accuracy: {}'.format(acc))
print('Train F1 Score: {}'.format(f1))

Train Accuracy: 0.7343117408906883
Train F1 Score: [0.74201474 0.72613459]


### 전처리 후 데이터

In [58]:
model = BernoulliNB()
model.fit(X_train_scale, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [59]:
predict = model.predict(X_train_scale)
acc = metrics.accuracy_score(y_train, predict)
f1 = metrics.f1_score(y_train, predict, average=None)

print("Train Accuracy: {}".format(acc))
print("Train F1 score {}".format(f1))

Train Accuracy: 0.7227635075287865
Train F1 score [0.72149485 0.72402066]


In [60]:
predict = model.predict(X_test_scale)
acc = metrics.accuracy_score(y_test, predict)
f1 = metrics.f1_score(y_test, predict, average=None)

print("Train Accuracy: {}".format(acc))
print("Train F1 score {}".format(f1))

Train Accuracy: 0.7343117408906883
Train F1 score [0.74427667 0.7235387 ]


## 다항 나이브 베이즈

* 입력 특성이 다항분포에 의해 생성된 빈도수 값을 갖는 다고 가정

### 전처리를 거치지 않은 데이터

In [63]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

In [64]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [67]:
predict = model.predict(X_train)
acc = metrics.accuracy_score(y_train, predict)
f1 = metrics.f1_score(y_train, predict, average=None)

print('Train Accuracy: {}'.format(acc))
print('Train F1 Score: {}'.format(f1))

Train Accuracy: 0.6692395292926737
Train F1 Score: [0.66504357 0.67333167]


In [68]:
predict = model.predict(X_test)
acc = metrics.accuracy_score(y_test, predict)
f1 = metrics.f1_score(y_test, predict, average=None)

print('Train Accuracy: {}'.format(acc))
print('Train F1 Score: {}'.format(f1))

Train Accuracy: 0.7358299595141701
Train F1 Score: [0.75652985 0.71128319]
